In [1]:
from currensee.agents.complete_graph import compiled_graph
from currensee.agents.tools.finance_tools import (generate_macro_table,
    retrieve_client_industry_news,
    retrieve_holdings_news,
    retrieve_macro_news,
    summarize_finance_outputs)
    
import joblib
import warnings
warnings.filterwarnings("ignore")
from currensee.utils.output_utils_dynamic import (
    generate_report,
    format_news_summary_to_html,
    format_paragraph_summary_to_html,
    save_html_to_file,
    convert_html_to_pdf,
)


warnings.filterwarnings("ignore")



2025-07-31 23:28:47,866 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-07-31 23:28:47,872 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-07-31 23:28:47,873 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


In [2]:
from datetime import date

today = date.today()
today_date = today.strftime("%Y%m%d")
print(today_date)

20250731


In [5]:
#from google.cloud import secretmanager
#import pandas as pd
#from currensee.core.settings import Settings
#db_name = "crm_outlook"
#engine = create_pg_engine(db_name)
#settings = Settings()
#from currensee.utils.db_utils import create_pg_engine


#df = pd.read_sql("SELECT * from email_data", con=engine)
#df.head(10)

In [10]:
#print(df[['to_emails', 'to_names']].drop_duplicates().sort_values(by='to_emails'))

                                      to_emails          to_names
12                        adam.clay@compass.com         Adam Clay
15                 amy.winters@gamestopcorp.com       Amy Winters
44                     anna.lawrence@matson.com     Anna Lawrence
0                      cynthia.hobbs@abbvie.com     Cynthia Hobbs
37               david.moreno@manpowergroup.com      David Moreno
9                    denise.moore@celestica.com      Denise Moore
1                  jane.moneypenny@bankwell.com   Jane Moneypenny
3             jennifer.phelps@aerovironment.com   Jennifer Phelps
21                    jessica.palmer@hasbro.com    Jessica Palmer
50        kelly.smith@presidiopropertytrust.com       Kelly Smith
6                      kyle.waters@amedisys.com       Kyle Waters
34   lisa.kennedy@lockheedmartincorporation.com      Lisa Kennedy
41                    mary.vasquez@marriott.com      Mary Vasquez
28       michelle.jenkins@intuitivesurgical.com  Michelle Jenkins
18        

In [ ]:
#df = pd.read_sql("""SELECT p.as_of_date
#        , p.employee_first_name
#        , p.employee_last_name
#        , p.finance_detail
#        , p.news_detail
#        , p.macro_news_detail
#        , p.past_meeting_detail 
#        , p.email	
#        FROM preferences p
#        where p.email = 'jane.moneypenny@bankwell.com'
#        """, con=engine)
#df.head(10)

In [ ]:
#query_str = f"""
#    SELECT  meeting_timestamp
#    FROM  meeting_data
#    WHERE  invitee_emails = 'adam.clay@compass.com'
#    and host_email = 'jane.moneypenny@bankwell.com'
#    and meeting_timestamp < '2024-03-26 11:00:00'
#    ORDER BY meeting_timestamp DESC
    #
#    LIMIT 1;"""


In [6]:
#from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper

#search = GoogleSerperAPIWrapper()

# Simple test query
#query = "macroeconomic news from reuters, bloomberg, and wsj"
#results = search.run(query)

# Print results
#print(results)

The latest news on the economy, jobs market, U.S. trade and housing, including the latest news and analysis on the Federal Reserve and Central Banking. Bloomberg delivers business and markets news, data, analysis, and video to the world, featuring stories from Businessweek and Bloomberg News. Federal Reserve officials expect they will need to resume lowering interest rates eventually—they just aren't ready to do so Wednesday. Top News ; Americas · Canada's GDP shrinks in May, could avoid contraction in second quarter 13 hours ago ; United States · US weekly jobless claims rise marginally ... Missing: bloomberg, wsj. The economy has motored through a swift run-up in interest rates in 2022, a regional banking crisis in 2023, and now trade disruptions in 2025, ... Check economic calendar events in real time on Bloomberg and see the global financial market impact by country and importance with previous, forecast and ... Inflation expectations are under control, private-sector hiring is wea

In [3]:
#from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper

#def retrieve_macro_news_simple():
#    try:
#        search = GoogleSerperAPIWrapper(k=5, serper_api_key="62c2f16ed75ff190d77686b02ef97cfa5b1f616c")  
#        query = "latest macroeconomic news site:bloomberg.com OR site:reuters.com"
#        print("Query:", query)
#        results = search.results(query)

#        if results.get("organic"):
#            for r in results["organic"][:5]: 
#                print("Title:", r.get("title"))
#                print("Link:", r.get("link"))
#                print("Snippet:", r.get("snippet"))
#                print("------")
#        else:
#            print("No organic results found.")
#    except Exception as e:
#        print("Search failed:", e)

# Run it
#retrieve_macro_news_simple()

Query: latest macroeconomic news site:bloomberg.com OR site:reuters.com
Title: Econ World | News Reshaping The Global Economy
Link: https://www.reuters.com/markets/econ-world/
Snippet: Top News ; Americas · Canada's GDP shrinks in May, could avoid contraction in second quarter 13 hours ago ; United States · US weekly jobless claims rise marginally ...
------
Title: Bloomberg - Business News, Stock Markets, Finance ...
Link: https://www.bloomberg.com/
Snippet: US Stocks Head for Another Record High Ahead of Fed Meeting · Even Top Earners Are Falling Behind on Credit Card and Car Payments · Chile's Central Bank Is Set to ...
------
Title: Economics
Link: https://www.bloomberg.com/economics-v2
Snippet: Get the latest US & global economy news. Read about economic policy, jobs, trade, banks, inflation, taxes, and more.
------
Title: Global Market Headlines | Breaking Stock Market News
Link: https://www.reuters.com/markets/
Snippet: Find the latest stock market news from every corner of the 

In [19]:
from langchain_community.utilities import GoogleSerperAPIWrapper
def retrieve_macro_news(state: dict) -> dict:
    from datetime import datetime

    def format_google_date(date_str):
        return datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d")

    start_date = state.get("last_meeting_timestamp")
    end_date = state.get("meeting_timestamp")
    google_start = format_google_date(start_date)
    google_end = format_google_date(end_date)
    sort_param = f"date:r:{google_start}:{google_end}"
    print(f"Calling API with sort param: {sort_param}")

    # Try a simple query for now
    filled_query = '"macroeconomic news" site:reuters.com'
    print(f"Running query: {filled_query}")

    search = GoogleSerperAPIWrapper(
        k=30,
        serper_api_key="62c2f16ed75ff190d77686b02ef97cfa5b1f616c",
        sort=sort_param
    )
    results = search.results(filled_query)

    if results.get("organic"):
        macro_summary = [r.get("title", "") for r in results.get("organic", [])]
    else:
        macro_summary = "No results found."

    state["macro_news_sources"] = macro_summary
    return state


In [ ]:
def retrieve_client_industry_news(state: dict) -> dict:
    client_company = state["client_company"]
    industry = state["client_industry"]

    site_filter_str = " OR ".join(f"site:{site}" for site in allowed_sites)
    
    # Simpler query without parentheses or quotes
    filled_query = f'{site_filter_str} {client_company} {industry} industry'
    
    print("Running query:", filled_query)
    
    search = GoogleSerperAPIWrapper(k=5, serper_api_key="62c2f16ed75ff190d77686b02ef97cfa5b1f616c")
    results = search.results(filled_query)
    
    if results.get("organic"):
        sorted_results_client = sorted(
            results.get("organic", []), key=score_result, reverse=True
        )
        industry_summary = sorted_results_client
    else:
        industry_summary = "No results found for client or industry news."
    
    new_state = state.copy()
    new_state["client_industry_sources"] = industry_summary
    return new_state


In [5]:
test_state = {
    "last_meeting_timestamp": "2024-06-16 11:00:00",
    "meeting_timestamp": "2025-07-30 09:00:00",
    "client_company": "Compass",
    "client_industry": "Real Estate"
}

print(retrieve_client_industry_news(test_state))

DEBUG: Filtering date range is from 2024-06-16 to 2025-07-30
DEBUG: Sending query to API: 'site_filter news about Compass and about Real Estate industry'

DEBUG: --- Raw API Results (before filtering) ---
[{'attributes': {'Missing': 'site_filter | Show results with:site_filter'},
  'date': '22 hours ago',
  'link': 'https://www.realestatenews.com/2025/07/30/compass-reports-best-ever-quarter-takes-aim-at-portals',
  'position': 1,
  'snippet': 'Revenue was up 21.1%, profits nearly doubled and agent count '
             'surged in Q2. CEO Robert Reffkin also took time to blast portals '
             'and MLSs over ...',
  'title': 'Compass reports best-ever quarter, takes aim at portals'},
 {'link': 'https://www.compass.com/newsroom/press-releases/',
  'position': 2,
  'snippet': 'Compass leads 2025 RealTrends The Thousand List, representing '
             '18% of the sales volume category – the most of any brokerage.',
  'title': 'Compass News: Announcements & Press'},
 {'attributes': {

In [3]:
test_state_macro = {
    "last_meeting_timestamp": "2024-01-16 11:00:00",
    "meeting_timestamp": "2025-07-30 09:00:00",
}

print(retrieve_macro_news(test_state_macro))

DEBUG: Filtering macro news from 2024-01-16 to 2025-07-30
DEBUG: Sending macro query to API: 'site_filter news about relevant macro events and the economy'

DEBUG: --- Raw Macro API Results (before filtering) ---
[{'attributes': {'Missing': 'site_filter | Show results with:site_filter'},
  'link': 'https://www.cnbc.com/economy/',
  'position': 1,
  'sitelinks': [{'link': 'https://www.cnbc.com/world-economy/',
                 'title': 'World Economy'},
                {'link': 'https://www.cnbc.com/2025/05/09/germanys-economy-chief-reiche-sets-out-roadmap-to-end-turmoil.html',
                 'title': "Germany's economy chief..."},
                {'link': 'https://www.cnbc.com/federal-reserve/',
                 'title': 'Federal Reserve'},
                {'link': 'https://www.cnbc.com/central-banks/',
                 'title': 'Central Banks'}],
  'snippet': 'Find the latest economic news, current events and headlines, as '
             'well as blogs and video from CNBC.com.',
  '

In [3]:
test_state_holdings = {
    "last_meeting_timestamp": "2024-06-16 11:00:00",
    "meeting_timestamp": "2025-07-30 09:00:00",
    "client_holdings": ["Apple", "Microsoft", "Tesla"] 
}

print(retrieve_holdings_news(test_state_holdings))

DEBUG: Filtering date range is from 2024-06-16 to 2025-07-30
DEBUG: Sending query for holding 'Apple': 'site_filter news about any of these top holdings:Apple'

DEBUG: --- Raw API Results for 'Apple' ---
[{'attributes': {'Missing': 'site_filter | Show results with:site_filter'},
  'link': 'https://finance.yahoo.com/quote/AAPL/news/',
  'position': 1,
  'snippet': '↗️ Apple (AAPL): Shares rose about 1.6% after the company '
             "reported iPhone sales blew past Wall Street's expectations. The "
             'Wall Street Journal • 27 minutes ago.',
  'title': 'Apple Inc. (AAPL) Latest Stock News & Headlines - Yahoo Finance'},
 {'attributes': {'Missing': 'site_filter | Show results with:site_filter'},
  'link': 'https://finance.yahoo.com/quote/AAPL/holders/',
  'position': 2,
  'snippet': 'Top Institutional Holders ; Vanguard Group Inc, 1.4B · Mar 31, '
             '2025, 9.38%, 290,762,158,483 ; Blackrock Inc. 1.14B · Mar 31, '
             '2025, 7.63%, 236,671,918,076.',
  'ti

TypeError: score_result() missing 1 required positional argument: 'keywords'

In [7]:
generate_macro_table()

,Indicator,Level,1-Month Change (%),3-Month Change (%),6-Month Change (%),1-Year Change (%),2-Year Change (%)
0,S&P 500 Index,6362.90,2.66,14.25,5.36,16.46,40.23
1,WTI Crude Oil Price,67.81,1.73,7.12,-7.75,-13.33,-10.07
2,US Dollar Index (Broad),120.41,0.29,-2.3,-6.14,-2.8,1.78
3,US CPI,321.50,4.5,None,None,None,None
4,US GDP Growth Rate,3.00,,,,,
5,US Unemployment Rate,4.10,,,,,
6,10-Year Treasury Yield,4.38,,,,,
7,Fed Funds Rate,4.33,,,,,


## Demo Case 1 - Compass

In [4]:

init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Adam Clay",
    "client_email": "adam.clay@compass.com",
    "meeting_timestamp": "2025-06-16 11:00:00",
    "meeting_description": "Compass - AML Requirements & Policy Developments",
}

result = compiled_graph.invoke(init_state)
#summary_client_comms = result["summary_client_comms"]
#fin_hold_summary_sourced= result["fin_hold_summary_sourced"]
#client_news_summary_sourced= result["client_news_summary_sourced"]
# full_report = summary + "\n\n### Macro Financial Snapshot\n\n" + macro_table
#print("Client Communications Summary: ", summary_client_comms)
#print("------------------------------------")
#print("Financial Holdings News Summary: ", fin_hold_summary_sourced)
#print("------------------------------------")
#print("Client Company News Summary: ", client_news_summary_sourced)

DEBUG: Filtering macro news from 2025-05-25 to 2025-06-16
DEBUG: Sending macro query to API: 'site_filter news about relevant macro events and the economy'

DEBUG: --- Raw Macro API Results (before filtering) ---
[{'attributes': {'Missing': 'site_filter | Show results with:site_filter'},
  'link': 'https://www.cnbc.com/economy/',
  'position': 1,
  'sitelinks': [{'link': 'https://www.cnbc.com/world-economy/',
                 'title': 'World Economy'},
                {'link': 'https://www.cnbc.com/2025/05/09/germanys-economy-chief-reiche-sets-out-roadmap-to-end-turmoil.html',
                 'title': "Germany's economy chief..."},
                {'link': 'https://www.cnbc.com/federal-reserve/',
                 'title': 'Federal Reserve'},
                {'link': 'https://www.cnbc.com/central-banks/',
                 'title': 'Central Banks'}],
  'snippet': 'Find the latest economic news, current events and headlines, as '
             'well as blogs and video from CNBC.com.',
  '

ValueError: One or more finance tool outputs are empty or missing. Cannot summarize.

In [5]:

init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Jessica Palmer",
    "client_email": "jessica.palmer@hasbro.com",
    "meeting_timestamp": "2025-07-10 14:30:00",
    "meeting_description": "Hasbro - Discuss Portfolio Diversification and Risk Strategy",
}

result = compiled_graph.invoke(init_state)

DEBUG: Filtering macro news from 2025-06-29 to 2025-07-10
DEBUG: Sending macro query to API: 'site_filter news about relevant macro events and the economy'

DEBUG: --- Raw Macro API Results (before filtering) ---
[{'attributes': {'Missing': 'site_filter | Show results with:site_filter'},
  'link': 'https://www.cnbc.com/economy/',
  'position': 1,
  'sitelinks': [{'link': 'https://www.cnbc.com/world-economy/',
                 'title': 'World Economy'},
                {'link': 'https://www.cnbc.com/2025/05/09/germanys-economy-chief-reiche-sets-out-roadmap-to-end-turmoil.html',
                 'title': "Germany's economy chief..."},
                {'link': 'https://www.cnbc.com/federal-reserve/',
                 'title': 'Federal Reserve'},
                {'link': 'https://www.cnbc.com/central-banks/',
                 'title': 'Central Banks'}],
  'snippet': 'Find the latest economic news, current events and headlines, as '
             'well as blogs and video from CNBC.com.',
  '

TypeError: score_result() missing 1 required positional argument: 'keywords'

In [4]:
# Run this line will save the result
joblib.dump(result, 'test1.joblib')

['test1.joblib']

In [5]:
result = joblib.load("test1.joblib")

In [4]:
result

{'user_email': 'jane.moneypenny@bankwell.com',
 'client_name': 'Adam Clay',
 'client_email': 'adam.clay@compass.com',
 'meeting_timestamp': '2024-03-26 11:00:00',
 'meeting_description': 'Compass - Annual Credit Facility Review Meeting',
 'client_company': 'Compass',
 'client_holdings': ['Taiwan Semiconductor Manufacturing',
  'CRH PLC',
  'ASML Holding NV',
  'Samsung Electronics Co Ltd',
  'SAP SE'],
 'client_industry': 'RealEstate',
 'all_client_emails': ['adam.clay@compass.com', 'jason.ramos@compass.com'],
 'last_meeting_timestamp': '2021-10-01 11:00:00',
 'email_summary': "This meeting concerns the annual credit facility review for Compass.  Initial contact between Jane Moneypenny of Bankwell Financial and Adam Clay of Compass regarding Compass's need for expansion capital occurred at the Real Estate Finance conference.  Following a call scheduled for Tuesday morning, Jane Moneypenny sent Adam Clay an indicative term sheet for a $25M revolving line of credit.  After internal revie

In [7]:
default_client_report = generate_report(result)
save_html_to_file(default_client_report , 'default_client_report .html')

## Test Case 2 - Hasbro Portfolio Diversification

In [5]:
#init_state = {
#    "user_email" : "jane.moneypenny@bankwell.com",
#    "client_name": "Jessica Palmer",
#    "client_email": "jessica.palmer@hasbro.com",
#    "meeting_timestamp": "2024-02-04 14:30:00",
#    "meeting_description": "Hasbro - Discuss Portfolio Diversification and Bond Strategy",
#}

#result = compiled_graph.invoke(init_state)
#summary_client_comms = result["summary_client_comms"]
#fin_hold_summary_sourced= result["fin_hold_summary_sourced"]
#client_news_summary_sourced= result["client_news_summary_sourced"]
# full_report = summary + "\n\n### Macro Financial Snapshot\n\n" + macro_table
#print("Client Communications Summary: ", summary_client_comms)
#print("------------------------------------")
#print("Financial Holdings News Summary: ", fin_hold_summary_sourced)
#print("------------------------------------")
#print("Client Company News Summary: ", client_news_summary_sourced)

## Test Case 3 - Hasbro H1 FX Hedging Strategy

In [10]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Jessica Palmer",
    "client_email": "jessica.palmer@hasbro.com",
    "meeting_timestamp": "2023-12-11 11:00:00",
    "meeting_description": "Hasbro - Finalize H1 2024 FX Hedging Strategy",
}

result = compiled_graph.invoke(init_state)
#summary_client_comms = result["summary_client_comms"]
#fin_hold_summary_sourced= result["fin_hold_summary_sourced"]
#client_news_summary_sourced= result["client_news_summary_sourced"]
# full_report = summary + "\n\n### Macro Financial Snapshot\n\n" + macro_table
#print("Client Communications Summary: ", summary_client_comms)
#print("------------------------------------")
#print("Financial Holdings News Summary: ", fin_hold_summary_sourced)
#print("------------------------------------")
#print("Client Company News Summary: ", client_news_summary_sourced)


Generating report with the following preferences: 
 Holdings Detail: short 
 Client News Detail: full 
 Macro News Detail: none
 Past Meeting Detail: none



In [6]:
#result

In [11]:
#joblib.dump(result, 'test2.joblib')

['test2.joblib']

In [5]:
result = joblib.load("test2.joblib")

In [6]:
#result

In [6]:
client_report_auto = generate_report(result)
save_html_to_file(client_report_auto, 'client_report_auto.html')

## Test Case 4 - Ryan Holloway

In [10]:
init_state = {
    "user_email" : "ryan.holloway@bankwell.com",
    "client_name": "Adam Clay",
    "client_email": "adam.clay@compass.com",
    "meeting_timestamp": "2024-03-26 11:00:00",
    "meeting_description": "Compass - Annual Credit Facility Review Meeting",
}

result = compiled_graph.invoke(init_state)
#summary_client_comms = result["summary_client_comms"]
#fin_hold_summary_sourced= result["fin_hold_summary_sourced"]
#client_news_summary_sourced= result["client_news_summary_sourced"]
# full_report = summary + "\n\n### Macro Financial Snapshot\n\n" + macro_table
#print("Client Communications Summary: ", summary_client_comms)
#print("------------------------------------")
#print("Financial Holdings News Summary: ", fin_hold_summary_sourced)
#print("------------------------------------")
#print("Client Company News Summary: ", client_news_summary_sourced)


Generating report with the following preferences: 
 Finance Detail: none 
 News Detail: none 
 Macro News Detail: none
 Past Meeting Detail: short



In [11]:
#result

In [12]:
joblib.dump(result, 'test3.joblib')

['test2.joblib']

In [7]:
result = joblib.load("test3.joblib")

FileNotFoundError: [Errno 2] No such file or directory: 'test3.joblib'

In [14]:
client_report_sample3 = generate_report(result)
save_html_to_file(client_report_sample3, 'client_report_sample3.html')